# Лаб3. Рекомендательная система видеоконтента с implicit feedback – Spark ML
Задание: https://github.com/newprolab/sber-spark-ds-18/blob/main/labs/lab03.md

### Библиотеки и контекст

In [1]:
import findspark
findspark.init('/opt/spark-3.4.3/')

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.ml.feature import HashingTF, IDF
from pyspark.sql.types import *

import re

conf = SparkConf()
spark = (
    SparkSession
    .builder
    .config(conf=conf)
    .appName('max_burdasov_lab3')
    .getOrCreate()
)

sc = spark.sparkContext

https://repos.spark-packages.org/ added as a remote repository with the name: repo-1
Ivy Default Cache set to: /data/home/maksim.burdasov/.ivy2/cache
The jars for the packages stored in: /data/home/maksim.burdasov/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a4b78f56-9c15-4f0a-a7f9-d003e3bc4fe8;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/spark-3.4.3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.4.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.4.3 in central
	found org.apache.kafka#kafka-clients;3.3.2 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.6 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 489ms :: artifacts dl 10ms
	:: modules in use:
	com.google.code.findbugs#jsr305;3.0.0 from central in [default]
	commons-logging#commons-logging;1.1.3 from central in [default]
	org.apache.commons#commons-pool2;2.11.1 from central in [default]
	org.apache.hadoop#hadoop-client-api;3.3.4 from central in [default]
	org.apache.hadoop#

### Данные

In [79]:
!hdfs dfs -ls /labs/slaba03/

Found 4 items
-rw-r--r--   3 hdfs hdfs   91066524 2022-01-06 18:46 /labs/slaba03/laba03_items.csv
-rw-r--r--   3 hdfs hdfs   29965581 2022-01-06 18:46 /labs/slaba03/laba03_test.csv
-rw-r--r--   3 hdfs hdfs   74949368 2022-01-06 18:46 /labs/slaba03/laba03_train.csv
-rw-r--r--   3 hdfs hdfs  871302535 2022-01-06 18:46 /labs/slaba03/laba03_views_programmes.csv


In [3]:
# Таргеты
train_df = (
    spark
    .read
    .format('csv')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .load('/labs/slaba03/laba03_train.csv')
)

train_df = train_df.cache()
train_df.show(3)

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|  74107|       0|
|   1654|  89249|       0|
|   1654|  99982|       0|
+-------+-------+--------+
only showing top 3 rows



In [4]:
# Данные о передачах
items_df = (
    spark
    .read
    .format('csv')
    .option("delimiter", "\t")
    .option('header', 'true')
    .option('inferSchema', 'true')
    .load('/labs/slaba03/laba03_items.csv')
)

items_df = items_df.cache()
items_df.show(1, truncate=False, vertical=True)

-RECORD 0--------------------------------------------------------------------
 item_id                     | 65667                                         
 channel_id                  | null                                          
 datetime_availability_start | 1970-01-01 03:00:00                           
 datetime_availability_stop  | 2018-01-01 03:00:00                           
 datetime_show_start         | null                                          
 datetime_show_stop          | null                                          
 content_type                | 1                                             
 title                       | на пробах только девушки (all girl auditions) 
 year                        | 2013.0                                        
 genres                      | Эротика                                       
 region_id                   | null                                          
only showing top 1 row



In [5]:
# Дополнительная информация
views_programmes_df = (
    spark
    .read
    .format('csv')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .load('/labs/slaba03/laba03_views_programmes.csv')
)

views_programmes_df = views_programmes_df.cache()
views_programmes_df.show(15, truncate=False)

+-------+-------+----------+----------+---------+
|user_id|item_id|ts_start  |ts_end    |item_type|
+-------+-------+----------+----------+---------+
|0      |7101053|1491409931|1491411600|live     |
|0      |7101054|1491412481|1491451571|live     |
|0      |7101054|1491411640|1491412481|live     |
|0      |6184414|1486191290|1486191640|live     |
|257    |4436877|1490628499|1490630256|live     |
|1654   |7489015|1493434801|1493435401|live     |
|1654   |7489023|1493444101|1493445601|live     |
|1654   |6617053|1489186156|1489200834|live     |
|1654   |6438693|1487840070|1487840433|live     |
|1654   |6526859|1488705452|1488706154|live     |
|1654   |6526754|1488532396|1488532895|pvr      |
|1654   |6239098|1486732011|1486732410|live     |
|1654   |6438763|1488305761|1488307286|pvr      |
|1654   |7489013|1493433301|1493434201|live     |
|1654   |6317094|1486829784|1486830389|live     |
+-------+-------+----------+----------+---------+
only showing top 15 rows



In [6]:
# Данные для инференса
infer_df = (
    spark
    .read
    .format('csv')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .load('/labs/slaba03/laba03_test.csv')
)

infer_df = infer_df.cache()
infer_df.show(1)

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|  94814|    null|
+-------+-------+--------+
only showing top 1 row



### Препроцессинг
Обработка данных и подготовка фичей для финального датасета

In [ ]:
"""
TODO: подготовить фичи из сырых данных
* фичи о программах (айтемах):
    + количество покупок этого айтема
    + количество записей по этому айтему в датафрейме
    + доля покупок этого айтема из всех записей по этому айтому
    + доля покупок этого айтема из всех покупок
    + лет с момента выпуска
    - смотрит ли пользователь этот канал
* фичи о пользователях
    + общее количество покупок пользователя
    + доля платных программ, которые пользовать приобрел
    + отношение количества покупок пользователя к среднему количествую покупок
    + перцентиль пользователя по количеству покупок
* фичи о пристрастиях пользователей (разделить на записанные и просмотренные в лайве)
    - общее количество сеансов
    - общее время просмотра
    - среднее время просмотра
    - среднее время перерывов между просмотрами
    - времени с первой просмотренной программы до текущего момента
    - времени просмотра платных программ
    - среднее время просмотра платных программ
    - доля просмотра платных программ
    - доля платных программ из всех уникальных отсмотренных программ
    - (множество колонок) время и доля просмотра каждого жанра от общих времени и количества просмотров
    - среднее количество лет с выпуска программ, которые он смотрит
    - топ 1 самый просматриваемый пользователем канал
    - топ 2 самый просматриваемый пользователем канал
    - топ 3 самый просматриваемый пользователем канал
    - регион, программы из которых больше всего смотрел
* кластера для программ
    + методом плеча определить оптимальное количество кластеров
    + выполнить кластеризацию по описаниям
    - (если останется время) добавить колонки с вероятностями принадлежности к кластерам
* (если останется время) кластеризация пользователей
"""

In [7]:
### Исследование данных

# # сколько всего пользователей (1941)
# train_df.select('user_id').distinct().count()

# # проверить, что у каждого пользователя одинакого строк (разное, но их от ~2400 до 2690)
# train_df.groupBy('user_id').count().agg({'count': 'max'}).show()

# # есть ли дубли в train_df (нет)
# print(train_df.count() - train_df.select('user_id', 'item_id').distinct().count())

# # сколько вообще покупок в таблице (10904 out of 5032624)
# ttl_cnt = train_df.count()
# prch_cnt = train_df.filter(F.col('purchase') == 1).count()
# print(ttl_cnt, prch_cnt)

# # уникальных программ (635'568)
# items_df.select('item_id').count()

# # всего платных программ (3704)
# items_df.filter(F.col('content_type') == 1).count()

# сколько программ в таблице train_df (3704, видимо, только платные и указаны)
# train_df.select('item_id').distinct().count()

# есть ли в таблице пользователи совсем без покупок (266 out of 1941)
# train_df.groupBy('user_id').agg(F.sum('purchase').alias('prch_cnt')).filter(F.col('prch_cnt') == 0).count()

# items_df.filter(F.col('channel_id').isNotNull()).count()  # 631864
# items_df.filter(F.col('channel_id').isNull()).count()  # 3704

# # сколько каналов определяется (207)
# items_df.select('channel_id').filter(F.col('channel_id').isNotNull()).distinct().count()

# # сколько платных передач, для которых не определен канал (все 3704)
# items_df.filter(
#     F.col('channel_id').isNull()
#     & (F.col('content_type') == 1)
# ).select('item_id').distinct().count()

In [8]:
### Расчет основных агрегат по покупкам

# Количество всех покупок всеми пользователями
ttl_prch_cnt, unq_users_cnt, unq_items_cnt = train_df.agg(
    F.sum('purchase'),  # всего покупок
    F.countDistinct('user_id'),  # уникальных пользователей
    F.countDistinct('item_id'),  # уникальных платных программ
).first()

user_avg_prch_cnt = ttl_prch_cnt / unq_users_cnt

print(
    f'Всего покупок: {ttl_prch_cnt}\n'
    f'Уникальных пользователей: {unq_users_cnt}\n'
    f'Уникальных платных программ: {unq_items_cnt}\n'
    f'Ср. кол-во покупок пользователем: {user_avg_prch_cnt}\n'
)

Всего покупок: 10904
Уникальных пользователей: 1941
Уникальных платных программ: 3704
Ср. кол-во покупок пользователем: 5.617722823286965



In [9]:
"""
Фичи о программах (айтемах):
    + item_id
    + количество покупок этого айтема
    + количество записей по этому айтему в датафрейме
    + доля покупок этого айтема из всех записей по этому айтому
    + доля покупок этого айтема из всех покупок
"""

### Фичи по покупаемости айтемов
items_buying_df = (
    train_df
    .groupBy('item_id')
    .agg(
        F.sum('purchase').alias('item_prch_cnt'),  # количество покупок этого айтема
        F.count('purchase').alias('item_rows_cnt'),  # количество записей по этому айтему
    )
)

items_buying_df = items_buying_df.withColumn(
    'item_prch_prc',  # доля покупок этого айтема из всех записей по этому айтому
    F.col('item_prch_cnt') * 100 / F.col('item_rows_cnt')
)

items_buying_df = items_buying_df.withColumn(
    'ttl_prhc_prc',  # доля покупок этого айтема из всех покупок
    F.col('item_prch_cnt') * 100 / F.lit(ttl_prch_cnt)
)

# Удаление технической колонки с количеством строк по этой программе
items_buying_df = items_buying_df.drop('item_rows_cnt')

items_buying_df = items_buying_df.cache()
items_buying_df.orderBy(F.desc('ttl_prhc_prc')).show(10, truncate=False)

+-------+-------------+------------------+------------------+
|item_id|item_prch_cnt|item_prch_prc     |ttl_prhc_prc      |
+-------+-------------+------------------+------------------+
|67036  |98           |7.153284671532846 |0.8987527512839325|
|90762  |83           |6.098457016899339 |0.7611885546588408|
|10585  |67           |4.988830975428146 |0.6144534115920763|
|8661   |67           |4.82361411087113  |0.6144534115920763|
|77442  |64           |4.698972099853157 |0.586940572267058 |
|74390  |63           |4.65288035450517  |0.5777696258253852|
|9180   |60           |4.4609665427509295|0.5502567865003668|
|9919   |59           |4.402985074626866 |0.5410858400586941|
|89637  |55           |4.0               |0.5044020542920029|
|9911   |55           |4.0087463556851315|0.5044020542920029|
+-------+-------------+------------------+------------------+
only showing top 10 rows



In [10]:
### Проверка
# items_buying.orderBy(F.desc('ttl_prhc_prc')).show()
# items_buying.agg(F.min('item_rows_cnt'), F.max('item_rows_cnt')).show()
# items_buying.filter(F.col('item_prch_cnt') != 0).count()

In [11]:
"""
Фичи о пользователях:
    + общее количество покупок пользователя
    + доля платных программ, которые пользовать приобрел
    + отношение количества покупок пользователя к среднему количествую покупок
    + перцентиль пользователя по количеству покупок
"""

### Фичи по покупателской способности пользователей
user_prch_df = (
    train_df
    .groupBy('user_id')
    .agg(
        F.sum('purchase').alias('user_prch_cnt'),  # количество покупок этого пользователя
        F.count('purchase').alias('user_rows_cnt'),  # количество записей по этому пользователю
    )
)

user_prch_df = user_prch_df.withColumn(
    'user_prch_prc',  # доля платных программ, которые пользовать приобрел
    F.col('user_prch_cnt') * 100 / F.col('user_rows_cnt')
)

user_prch_df = user_prch_df.withColumn(
    'user_prch_to_avg_ratio',  # отношение числа покупок пользователя к среднему числу покупок
    F.col('user_prch_cnt') / F.lit(user_avg_prch_cnt)
)

# Вычисление перцентиля для каждого пользователя
user_prch_df = user_prch_df.withColumn(
    'user_prch_percentile', 
    F.percent_rank().over(
        Window.orderBy("user_prch_cnt")
    )
)

# Удаление технической колонки с количеством строк по этому пользователю
user_prch_df = user_prch_df.drop('user_rows_cnt')

user_prch_df = user_prch_df.cache()
user_prch_df.show(5)

25/04/22 10:39:06 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+-------------+-------------+----------------------+--------------------+
|user_id|user_prch_cnt|user_prch_prc|user_prch_to_avg_ratio|user_prch_percentile|
+-------+-------------+-------------+----------------------+--------------------+
| 890476|            0|          0.0|                   0.0|                 0.0|
| 876234|            0|          0.0|                   0.0|                 0.0|
| 797350|            0|          0.0|                   0.0|                 0.0|
| 923662|            0|          0.0|                   0.0|                 0.0|
| 906790|            0|          0.0|                   0.0|                 0.0|
+-------+-------------+-------------+----------------------+--------------------+
only showing top 5 rows



In [12]:
### Проверка
# user_prch_df.agg(F.min('user_rows_cnt'), F.max('user_rows_cnt')).show()
# user_prch_df.filter(F.col('user_prch_cnt') > 60).show()

In [13]:
### Исследование данных
# items_df.show(1, truncate=False, vertical=True)
# items_df.select(F.countDistinct('datetime_availability_start')).show()
# items_df.groupBy('datetime_availability_stop').count().show()

# для скольких платных программ не указан год (для 4 из 3704)
# items_df.groupBy('genres').count().show(100)

# # для скольки платных программ указан канал (ни одна запись не содержит инфы о канале)
# #   при этом все бесплатные программы содержат инфу о канале
# items_df.filter((F.col('content_type') == 1) & F.col('channel_id').isNotNull()).count()

# для скольки платных программ указан регион (0)
# items_df.filter((F.col('content_type') == 1) & F.col('region_id').isNull()).count()

# # сколько уникальных регионов
# items_df.groupBy('region_id').count().show(100)

In [133]:
### Формирование бинарных колонок по жанрам

# Разделить жанры на отдельные значения в копии датафрейма
items_df = items_df.withColumn(
    'genres', 
    F.when(
        F.col('genres').isNull(), F.lit([])  # замена пропуска на пустой список
    ).otherwise(F.split('genres', ','))
)

# Список уникальных жанров
unq_genres = (
    items_df
    .filter(F.col('genres').isNotNull())
    .select('genres')
    .rdd.flatMap(lambda row: row[0])
    .distinct()
    .collect()
)

# Маппиг жанров вида {название_жанра : genre_n, ...}
genres_mapping = dict()
for unq_gnr in unq_genres:
    genres_mapping[unq_gnr] = f'genre_{len(genres_mapping)}'

# Создание бинарных колонок для каждого жанра
for gnr in genres_mapping:
    items_df = items_df.withColumn(
        f'{genres_mapping[gnr]}_flg',  # чтобы не использовать кириллицу, заменим название жанра на текст genre_n_flg 
        F.array_contains(F.col('genres'), gnr).cast("integer")
    )

# Получение возраста программы
items_df = items_df.withColumn(
    'item_age',  # возраст программы
    2025 - F.col('year').cast("integer")
)

# Малоинформативные и технические колонки
items_cols_to_drop = [
    'datetime_availability_start',
    'datetime_availability_stop',
    'datetime_show_start',
    'datetime_show_stop',
    'region_id',
    'genres',  # данные из колонки уже получены
    'year'  # данные из колонки уже получены
]

# Удаление колонок
items_df = items_df.drop(*items_cols_to_drop)

items_df = items_df.cache()
items_df.show(1)

In [134]:
### TF-IDF на колонке title

@F.pandas_udf(ArrayType(StringType()))
def get_tokens_list(text_series):
    """Преобразовать строки в списки токенов."""
    regex = re.compile(u'[\w\d]{2,}', re.U)  # паттерн для поиска слов
    tokens_series = text_series.str.lower().apply(lambda row: regex.findall(row) if row else [])
    return tokens_series


# Токенизация 'title'
items_df = items_df.withColumn('title_tokens', get_tokens_list('title'))

# Расчет TF с помощью HashingTF
hashing_tf = HashingTF(numFeatures=1024, inputCol="title_tokens", outputCol="tf_features")
items_df = hashing_tf.transform(items_df)

# Расчет IDF с помощью IDF
idf = IDF(inputCol='tf_features', outputCol='tfidf_features')
items_df = idf.fit(items_df).transform(items_df)

items_df = items_df.cache()
items_df.show(1)

In [16]:
# Удаление колонок
items_df = items_df.drop('title', 'title_tokens', 'tf_features')

In [18]:
# ### Поиск оптимального количества кластеров

# import numpy as np

# from pyspark.ml.clustering import KMeans
# from pyspark.ml.evaluation import ClusteringEvaluator

# def elbow_method(df, max_clusters=10):
#     wssse = []
    
#     for k in range(2, max_clusters + 1):  # Начинаем с 2 кластеров до максимального значения
#         kmeans = KMeans().setK(k).setFeaturesCol('tfidf_features').setPredictionCol('prediction')
#         model = kmeans.fit(df)
        
#         evaluator = ClusteringEvaluator(featuresCol='tfidf_features')
#         silhouette = evaluator.evaluate(model.transform(df))
        
#         cost = model.summary.trainingCost
#         wssse.append((k, cost))
    
#     return wssse

# def find_elbow_point(wssse):
#     """
#     Метод для нахождения точки перегиба на основе первой разности.
#     """
#     n_points = len(wssse)
#     all_knots = range(len(wssse))
#     knots_diff = np.diff(wssse[:, 1]) / np.diff(all_knots)  # Разница между соседними точками
#     second_diff = np.diff(knots_diff)  # Вторая разность
    
#     candidate_idxs = np.where(second_diff > 0)[0]  # Индекс точек, где вторая разность положительна
#     if not len(candidate_idxs):
#         print("Не найдено явной точки перегиба.")
#         return None
    
#     elbow_point = candidate_idxs[0] + 2  # Выбор первой точки перегиба
#     return elbow_point

# # DataFrame с колонкой SparseVector
# # tfidf_features - название колонки с векторами
# df = items_tfidf_df.select('tfidf_features')

# # Определяем оптимальное количество кластеров
# optimal_clusters = elbow_method(df, max_clusters=10)

# # Нахождение точки перегиба
# elbow_point = find_elbow_point(np.array(optimal_clusters))
# if elbow_point is not None:
#     optimal_k = elbow_point
#     print(f'Optimal K: {optimal_k}')
# else:
#     print("Точку перегиба не удалось определить автоматически.")

# # optimal_k: 3

In [135]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Модель KMeans
kmeans = (
    KMeans()
    .setK(3)
    .setSeed(42)
    .setFeaturesCol('tfidf_features')
    .setPredictionCol('prediction')
)

# Обучение модели
model = kmeans.fit(items_df)

# Прогнозирование и добавление кластеров к данным
items_df = model.transform(items_df)

In [136]:
result_items_df = items_df.drop('tfidf_features').cache()
result_items_df.show(1)

In [137]:
# views_programmes_df.show()

In [21]:
### Фичи из истории просмотров

# Продолжительность просмотра
hist_df = views_programmes_df.withColumn(
    'duration',
    F.col('ts_end') - F.col('ts_start')
)

hist_df = hist_df.drop('ts_start', 'ts_end')

hist_df = hist_df.cache()
hist_df.show(10)

+-------+-------+---------+--------+
|user_id|item_id|item_type|duration|
+-------+-------+---------+--------+
|      0|7101053|     live|    1669|
|      0|7101054|     live|   39090|
|      0|7101054|     live|     841|
|      0|6184414|     live|     350|
|    257|4436877|     live|    1757|
|   1654|7489015|     live|     600|
|   1654|7489023|     live|    1500|
|   1654|6617053|     live|   14678|
|   1654|6438693|     live|     363|
|   1654|6526859|     live|     702|
+-------+-------+---------+--------+
only showing top 10 rows



In [22]:
'''
Фичи по времени просмотра:
    + общему 
    + просмотру лайва
    + просмотру в записи
    + долю в записи от лайва
'''

hist_df_agg = (
    hist_df
    .groupBy('user_id')
    .agg(
        F.sum('duration').alias('ttl_duration'),
        F.sum(F.when(F.col('item_type') == 'live', F.col('duration')).otherwise(0)).alias('live_duration'),
        F.sum(F.when(F.col('item_type') == 'pvr', F.col('duration')).otherwise(0)).alias('pvr_duration'),
    )
)

hist_df_agg = hist_df_agg.withColumn(
    'pvr_ratio',
    F.when(F.col('live_duration') != 0, (F.col('pvr_duration') / F.col('live_duration')) * 100).otherwise(0)
)

hist_df_agg = hist_df_agg.cache()
hist_df_agg.show(1)

+-------+------------+-------------+------------+------------------+
|user_id|ttl_duration|live_duration|pvr_duration|         pvr_ratio|
+-------+------------+-------------+------------+------------------+
| 561425|       37699|        30959|        6740|21.770729028715397|
+-------+------------+-------------+------------+------------------+
only showing top 1 row



### Тренировочный датасет

In [138]:
dataset = (
    train_df
    .join(items_buying_df, how='left', on='item_id')
    .join(user_prch_df, how='left', on='user_id')
    .join(result_items_df, how='left', on='item_id')
    .join(hist_df_agg, how='left', on='user_id')
)

dataset = dataset.cache()
dataset.show(1)

In [24]:
dataset = dataset.withColumnRenamed('prediction', 'cluster_prediction')

In [112]:
# ### Проверка пустых значений
# null_counts = dataset.select([F.sum(F.col(c).isNull().cast("integer")).alias(c) for c in dataset.columns])
# null_counts.show(1, vertical=True)

In [98]:
# Категориальные колонки
feature_columns = [
    'item_prch_cnt', 'item_prch_prc', 'ttl_prhc_prc',
    'user_prch_cnt', 'user_prch_prc', 'user_prch_to_avg_ratio', 'user_prch_percentile',
    'content_type',
    'genre_0_flg', 'genre_1_flg', 'genre_2_flg', 'genre_3_flg', 'genre_4_flg', 'genre_5_flg',
    'genre_6_flg', 'genre_7_flg', 'genre_8_flg', 'genre_9_flg', 'genre_10_flg', 'genre_11_flg',
    'genre_12_flg', 'genre_13_flg', 'genre_14_flg', 'genre_15_flg', 'genre_16_flg', 'genre_17_flg',
    'genre_18_flg', 'genre_19_flg', 'genre_20_flg', 'genre_21_flg', 'genre_22_flg', 'genre_23_flg',
    'genre_24_flg', 'genre_25_flg', 'genre_26_flg', 'genre_27_flg', 'genre_28_flg', 'genre_29_flg',
    'genre_30_flg', 'genre_31_flg', 'genre_32_flg', 'genre_33_flg', 'genre_34_flg', 'genre_35_flg',
    'genre_36_flg', 'genre_37_flg', 'genre_38_flg', 'genre_39_flg', 'genre_40_flg', 'genre_41_flg', 
    'genre_42_flg', 'genre_43_flg', 'genre_44_flg', 'genre_45_flg', 'genre_46_flg', 'genre_47_flg', 
    'genre_48_flg', 'genre_49_flg', 'genre_50_flg', 'genre_51_flg', 'genre_52_flg', 'genre_53_flg', 
    'genre_54_flg', 'genre_55_flg', 'genre_56_flg', 'genre_57_flg', 'genre_58_flg', 'genre_59_flg', 
    'genre_60_flg', 'genre_61_flg', 'genre_62_flg', 'genre_63_flg', 'genre_64_flg', 'genre_65_flg', 
    'genre_66_flg', 'genre_67_flg', 'genre_68_flg', 'genre_69_flg', 'genre_70_flg', 'genre_71_flg',
    'genre_72_flg', 'genre_73_flg', 'genre_74_flg', 'genre_75_flg', 'genre_76_flg', 'genre_77_flg', 
    'genre_78_flg', 'genre_79_flg', 'genre_80_flg', 'genre_81_flg', 'genre_82_flg', 'genre_83_flg', 
    'item_age', 'cluster_prediction',
    'ttl_duration', 'live_duration', 'pvr_duration', 'pvr_ratio'
]

# Замена NaN на 0 в тренировочном датасете
for col_name in feature_columns:
    dataset = dataset.withColumn(
        col_name,
        F.when(F.isnan(F.col(col_name)), 0).otherwise(F.col(col_name))
    )

In [55]:
# dataset.dtypes
# dataset.show(1, False, True)

### Модель

In [139]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline


assembler = VectorAssembler(inputCols=feature_columns, outputCol="features", handleInvalid="skip")

# Инициализация классификатора
gbt = GBTClassifier(labelCol='purchase', featuresCol="features", maxIter=50, seed=42)

# Создаем pipeline (если нужны преобразования)
pipeline = Pipeline(stages=[assembler, gbt])

# Делим данные на обучающую и тестовую выборки (рекомендуется)
train_df, test_df = dataset.randomSplit([0.8, 0.2], seed=42)

# Обучение модели
model = pipeline.fit(train_df)

In [28]:
# BACKLOG: сделать сохранение через pickle для удобства (чтобы лежало локально)
# Сейчас какая-то ошибка всплывает при загрузке сохраненной модели.

# from pyspark.ml.classification import GBTClassificationModel

# # Сохранение модели 
# model_path = "./models/lab03_gbtc_model"
# model.write().overwrite().save(model_path)

# # Загрузка модели
# model = GBTClassificationModel.load("models/lab03_gbtc_model")

### Метрика ROC-AUC

In [141]:
# Предсказания на тестовой выборке
predictions = model.transform(test_df)

# Оценка модели
evaluator = BinaryClassificationEvaluator(labelCol="purchase", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = evaluator.evaluate(predictions)
print(f"Test AUC = {auc}")  

# # Пример: показать несколько предсказаний
# predictions.select("features", "purchase", "prediction", "probability").show(5)

Test AUC = 0.9376561339039494


### Инференс

In [142]:
inf_dataset = (
    infer_df
    .join(items_buying_df, how='left', on='item_id')
    .join(user_prch_df, how='left', on='user_id')
    .join(result_items_df, how='left', on='item_id')
    .join(hist_df_agg, how='left', on='user_id')
)

inf_dataset = inf_dataset.withColumnRenamed('prediction', 'cluster_prediction')

inf_dataset = inf_dataset.cache()
inf_dataset.count()

In [106]:
### Проверка соответсвия обучающего и инференс датасетов
# assert inf_dataset.columns == dataset.columns

In [116]:
# # Проверка пустых значений
# null_counts = inf_dataset.select([F.sum(F.col(c).isNull().cast("integer")).alias(c) for c in inf_dataset.columns])
# null_counts.show(1, vertical=True)

In [114]:
### Замена NaN на 0
for col_name in feature_columns:
    inf_dataset = inf_dataset.withColumn(
        col_name,
        F.when(F.col(col_name).isNull(), 0).otherwise(F.col(col_name))
    )

In [118]:
# Предсказания на тестовой выборке
inf_predicts = model.transform(inf_dataset)

In [119]:
inf_predicts.count()

2156840

In [120]:
### Получение итогового датасета с вероятностями покупки

from pyspark.ml.linalg import DenseVector, VectorUDT
from pyspark.sql.types import ArrayType, FloatType, DoubleType
import pandas as pd

# UDF для преобразования DenseVector в массив float
vector_to_array_udf = F.udf(lambda v: v.toArray().tolist() if v is not None else None, ArrayType(FloatType()))

# Преобразуем колонку probability в массивы
inf_res = inf_predicts.withColumn('probability_array', vector_to_array_udf('probability'))

# pandas_udf для извлечения второго элемента из массива
@F.pandas_udf(DoubleType())
def extract_second_element(vec_series: pd.Series) -> pd.Series:
    return vec_series.apply(lambda arr: float(arr[1]) if arr is not None and len(arr) > 1 else None)

# Применяем pandas_udf к новой колонке
inf_res = inf_res.withColumn('second_value', extract_second_element('probability_array'))

inf_res = inf_res.select(
    'user_id', 
    'item_id',
    F.col('second_value').alias('purchase')
)

inf_res = inf_res.cache()
inf_res.show(3)

25/04/22 20:39:43 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1230.3 KiB


+-------+-------+--------------------+
|user_id|item_id|            purchase|
+-------+-------+--------------------+
| 894161|   8389|0.022937152534723282|
| 894261|   8389| 0.02204175293445587|
| 894307|   8389| 0.02324526011943817|
+-------+-------+--------------------+
only showing top 3 rows



### Сохранение результатов

In [129]:
# sorted_res_df = (
#     inf_res
#     .orderBy(
#         F.asc('user_id'),
#         F.asc('item_id')
#     )
# )

In [143]:
# sorted_res_df.toPandas().to_csv("lab03.csv")

### Завершение контекста

In [145]:
spark.stop()